In [ ]:
pip install opencv-python roboflow ultralytics

In [ ]:
from ultralytics import YOLO
from roboflow import Roboflow

rf = Roboflow(api_key="Rdt********g0yxkVr6K")
project = rf.workspace("leo-ueno").project("people-detection-o4rdr")
version = project.version(9)
dataset = version.download("yolov8")

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
#we have cuda avaiable now we just have to shift it to the gpu from cpu
model=YOLO("yolov8s.pt")
device="cuda"
model.model.to(device)

In [ ]:
#train model
model.train(data=f"{dataset.location}/data.yaml", epochs=10, imgsz=640)

In [ ]:
import cv2
import time
from datetime import datetime
from google.colab.patches import cv2_imshow

cap = cv2.VideoCapture("people_walking.mp4")

if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
out = cv2.VideoWriter("output.mp4", cv2.VideoWriter_fourcc(*"mp4v"), fps, (width, height))

count = 0
batch = "batch"
batch_no = 1
balst = {}
processed_frames = 0
batch_interval_frames = int(30 * fps)
next_batch_frame = batch_interval_frames
current_display_time = datetime.now().strftime("%H:%M:%S")

while True:
    ret, frame = cap.read()
    if not ret:
        print(f"End of video or error reading frame after {processed_frames} frames.")
        break

    processed_frames += 1

    try:
        results = model.predict(source=frame, conf=0.25, iou=0.45)
    except Exception as e:
        print(f"Error in model.predict: {e}")
        results = None

    if processed_frames >= next_batch_frame:
        if count > 0:
            batch_start_frame = next_batch_frame - batch_interval_frames
            batch_end_frame = processed_frames
            batch_start_time = batch_start_frame / fps
            batch_end_time = batch_end_frame / fps
            batch_detection = str(round(batch_start_time)) + '-' + str(round(batch_end_time))
            balst[batch_detection] = count
            batch_no += 1
            count = 0
            current_display_time = datetime.now().strftime("%H:%M:%S")
        next_batch_frame += batch_interval_frames

    if results and len(results) > 0:
        count += len(results[0].boxes)
        box = results[0].plot()
    else:
        box = frame
    out.write(box)

if count > 0:
    batch_start_frame = next_batch_frame - batch_interval_frames
    batch_end_frame = processed_frames
    batch_start_time = batch_start_frame / fps
    batch_end_time = batch_end_frame / fps
    batch_detection = str(round(batch_start_time)) + '-' + str(round(batch_end_time))
    balst[batch_detection] = count

print(f"Total Batches: {len(balst)}")
print(f"Batch Details: {balst}")

cap.release()
out.release()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

print(balst)
categories=[]
count=[]
for key in balst:
  categories.append(key)
  count.append(balst[key]/903) #normalize to average people per frame
  #why 903 e.g., 18486 for 0–30 seconds sum of 903 frames

fig, ax = plt.subplots(figsize=(10, 5), layout='constrained')
ax.bar(categories, count)